#### train process
1. import csv to df
1. process df to be modelable
  1. add variables
    * dow (cat - 7 vars)
    * hr (cat - 24 vars)
        * pair dow with hr?
    * slow_day (cat - 0, 1)
    * fast_day (cat - 0, 1)
    * L2
        * log L2?
    * google_dist (needs id?)
    * weather (needs datetime?)
1. model
1. use cooksd + fox criterion to identify outliers
1. filter outliers and make new df

* need to clean hourly data (clean up nans)

## * 아래 2칸 실행

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [1]:
# 경로 변수 (고정)

dataLoc = '../_data/'

trainfile = 'train_03_997.csv'
old_trainfile = 'train.csv'
kaggle_trainfile = 'train_k.csv'

testfile = 'test_clean.csv'
old_testfile = 'test.csv'

train_171016 = '171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'


## `train_with_osrm_and_weather_hourly.csv` (전처리 전)

In [4]:
pd.options.display.max_columns = 40

In [16]:
# train_with_osrm_and_weather.csv
df_train = pd.read_csv(dataLoc + train_osrm_weather_hourly)
df_train.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,street_for_each_step,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,East 3rd Street|2nd Avenue|Chrystie Street|Del...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,North 7th Street|Roebling Street|North 10th St...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,8th Avenue|West 36th Street|1st Avenue|1st Ave...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,Gramercy Park North|East 21st Street|Park Aven...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,West 13th Street|Washington Street|West 14th S...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


### to add: dow, m_d, hr, L2, holiday, disaster
## * dow

In [21]:
s_dow = df_train['pickup_datetime'].apply(pd.to_datetime)

In [23]:
s_time = s_dow.copy()

In [30]:
s_dow = s_time.dt.weekday_name

In [32]:
df_train.insert(4, 'dow', s_dow)

In [33]:
df_train.head()

,id,vendor_id,Time,pickup_datetime,dow,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,street_for_each_step,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,Saturday,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,East 3rd Street|2nd Avenue|Chrystie Street|Del...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,Saturday,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,North 7th Street|Roebling Street|North 10th St...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,Saturday,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,8th Avenue|West 36th Street|1st Avenue|1st Ave...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,Saturday,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,Gramercy Park North|East 21st Street|Park Aven...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,Saturday,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,West 13th Street|Washington Street|West 14th S...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


In [34]:
df_copy = df_train

## * m_d

In [36]:
df_copy['pickup_datetime'] = df_copy['pickup_datetime'].apply(pd.to_datetime)

In [37]:
df_copy.insert(loc=4, column='m_d', value=np.vectorize(str)(df_copy['pickup_datetime'].dt.month*100
                                                             + df_copy['pickup_datetime'].dt.day))

In [38]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,street_for_each_step,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,East 3rd Street|2nd Avenue|Chrystie Street|Del...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,North 7th Street|Roebling Street|North 10th St...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,8th Avenue|West 36th Street|1st Avenue|1st Ave...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,Gramercy Park North|East 21st Street|Park Aven...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,West 13th Street|Washington Street|West 14th S...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * hr

In [39]:
df_copy.insert(loc=6, column='hr', value=df_copy['pickup_datetime'].dt.hour)

In [40]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,hr,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,number_of_steps,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,17,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,15,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,13,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,6,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,10,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * L2

In [41]:
def dist(plng, plat, dlng, dlat):
    return ((dlng-plng)**2 + (dlat-plat)**2) ** 0.5

In [42]:
df_copy.insert(loc=7, column='L2', value=np.vectorize(dist)(\
                df_copy['pickup_longitude'], df_copy['pickup_latitude'],\
                df_copy['dropoff_longitude'], df_copy['dropoff_latitude']))

df_copy.head()

,id,vendor_id,Time,pickup_datetime,m_d,dow,hr,L2,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,total_travel_time,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.069424,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,861.2,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.057791,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,786.3,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,430,Saturday,23,0.066698,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,703.9,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,0.018223,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,298.0,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,430,Saturday,23,0.027198,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,458.2,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * holiday

In [51]:
holidays = {(2016, 1, 1) : 'New Year\'s',
            (2016, 1, 18) : 'MLK',
            (2016, 2, 15) : 'Washington',
            (2016, 5, 30) : 'Memorial' }

def is_holiday(timestamp):
    return (timestamp.year, timestamp.month, timestamp.day) in holidays

t = pd.Timestamp(2016, 5, 30)

print(is_holiday(t))

True


In [53]:
s_hol = df_copy['pickup_datetime'].apply(is_holiday)
s_hol.sum()

15583

In [64]:
df_copy.insert(loc=4, column='is_hol', value=s_hol)

In [67]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,is_hol,m_d,dow,hr,L2,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,starting_street,end_street,total_distance,...,distance_per_step,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.069424,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,N,1454,East 3rd Street,Sherman Street,10010.9,...,195.7|229|437.7|102.6|12.3|488.4|2311.3|1494.7...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.057791,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,N,1409,North 7th Street,East 83rd Street,10357.0,...,264.2|238.3|713|319.8|629.2|1747.4|1285.8|549....,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.066698,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,N,1081,8th Avenue,East 93rd Street,9275.8,...,1457.9|1924|350.4|16.8|457.9|120.6|409.2|3653....,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.018223,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,N,800,Gramercy Park North,West 37th Street,2726.9,...,65.8|76.6|955.9|326.7|1301.8|0,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.027198,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,N,1151,West 13th Street,Avenue A,3455.8,...,78.8|87.7|702.9|608.4|34.4|367.5|304.2|1151.6|...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


## * disaster

??

In [69]:
df_copy.to_csv('../_data/train_with__all_data_no_disaster.csv', index=False)

---

In [ ]:
# p_lng is idx2
# ...
# d_lat is idx5

filtered_df = taxi_df

for i in range(2, 6):
    lower = np.percentile(taxi_df.iloc[:, i], 0.3)
    upper = np.percentile(taxi_df.iloc[:, i], 99.7)
    filtered_df = filtered_df[ (filtered_df.iloc[:, i] > lower) \
                         & (filtered_df.iloc[:, i] < upper) ]

filtered_df.shape
  

In [70]:
pd.options.display.max_columns = 60

In [2]:
df_copy = pd.read_csv('../_data/train_with__all_data_no_disaster.csv')

In [3]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,is_hol,m_d,dow,hr,L2,dropoff_datetime,...,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.069424,5/1/16 0:24,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.057791,5/1/16 0:22,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,False,430,Saturday,23,0.066698,5/1/16 0:17,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.018223,5/1/16 0:12,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,False,430,Saturday,23,0.027198,5/1/16 0:17,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


In [4]:
droppable0 = ['id', 'vendor_id', 'Time', 'pickup_datetime', 'dropoff_datetime',\
             'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',\
             'store_and_fwd_flag', 'starting_street', 'end_street',\
             'number_of_steps', 'street_for_each_step', 'distance_per_step', 'travel_time_per_step',\
             'step_maneuvers', 'step_direction', 'step_location_list',\
             'average temperature', 'Wind Dir', 'Events', 'Conditions']

droppable = ['id', 'vendor_id', 'Time', 'pickup_datetime', 'dropoff_datetime',\
             'store_and_fwd_flag', 'starting_street', 'end_street',\
             'number_of_steps', 'street_for_each_step', 'distance_per_step', 'travel_time_per_step',\
             'step_maneuvers', 'step_direction', 'step_location_list',\
             'average temperature', 'Wind Dir', 'Events', 'Conditions']

In [5]:
for col in droppable:
    del df_copy[col]

In [6]:
df_copy.head()

,is_hol,m_d,dow,hr,L2,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,minimum temperature,precipitation,snow fall,snow depth,Temp.,Humidity,Dew Point,Wind Speed,Gust Speed,Precip
0,False,430,Saturday,23,0.069424,1,-73.987793,40.724792,-73.975616,40.656445,...,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,-73.957596,40.717770,-73.951424,40.775230,...,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,-74.000954,40.742031,-73.947708,40.782200,...,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,-73.985733,40.738258,-73.993179,40.754890,...,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,-74.006615,40.740650,-73.985619,40.723362,...,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [7]:
df_copy.to_csv('../_data/train__dropped__lnglat_saved__no_disaster.csv', index=False)

In [8]:
df_copy.columns

Index(['is_hol', 'm_d', 'dow', 'hr', 'L2', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'trip_duration', 'total_distance',
       'total_travel_time', 'maximum temperature', 'minimum temperature',
       'precipitation', 'snow fall', 'snow depth', 'Temp.', 'Humidity',
       'Dew Point', 'Wind Speed', 'Gust Speed', 'Precip'],
      dtype='object')

In [35]:
df_copy = pd.read_csv('../_data/train__dropped__lnglat_saved__no_disaster.csv')

In [38]:
short_cols = ['is_hol', 'm_d', 'dow', 'hr', 'L2', 'pas_cnt', 'p_lng', 'p_lat', 'd_lng', 'd_lat', 'dur',
       'total_dist', 'total_time', 'day_max_temp',
       'day_min_temp', 'day_rainfall', 'day_snowfall', 'day_snowdepth',
       'hr_temp', 'hr_humid', 'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip']

In [39]:
df_copy.columns = short_cols

In [36]:
pd.options.display.max_columns = 60

In [40]:
df_copy.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_lng,p_lat,d_lng,d_lat,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,-73.987793,40.724792,-73.975616,40.656445,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,-73.957596,40.717770,-73.951424,40.775230,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,-74.000954,40.742031,-73.947708,40.782200,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,-73.985733,40.738258,-73.993179,40.754890,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,-74.006615,40.740650,-73.985619,40.723362,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [43]:
df_copy['m_d'] = df_copy['m_d'].apply(str)

In [41]:
df_copy.isnull().sum()

is_hol              0
m_d                 0
dow                 0
hr                  0
L2                  0
pas_cnt             0
p_lng               0
p_lat               0
d_lng               0
d_lat               0
dur                 0
total_dist          0
total_time          0
day_max_temp        0
day_min_temp        0
day_rainfall        0
day_snowfall        0
day_snowdepth       0
hr_temp          5293
hr_humid         5293
hr_dewpt         5293
hr_windspeed     5293
hr_gustspeed     5293
hr_precip        5293
dtype: int64

In [44]:
nans = df_copy.loc[df_copy['hr_temp'].isnull(), ['m_d', 'hr']].copy()
nans['m_d'] = nans['m_d'].apply(int)
print(nans.head())
nansarray = np.array(nans)
np.unique(nansarray, axis=0)

       m_d  hr
67827  419  11
67828  419  11
67829  419  11
67830  419  11
67831  419  11


array([[313,   4],
       [105,   0],
       [105,   1],
       [105,   2],
       [105,   3],
       [108,  10],
       [401,   8],
       [401,   9],
       [401,  10],
       [401,  11],
       [401,  12],
       [401,  13],
       [401,  14],
       [402,   8],
       [403,   0],
       [403,   1],
       [404,   0],
       [404,   2],
       [409,   6],
       [413,  22],
       [413,  23],
       [416,   0],
       [419,  11],
       [202,  23]], dtype=int64)

In [45]:
nan_pairs = np.unique(nansarray, axis=0)

nan_pairs

array([[313,   4],
       [105,   0],
       [105,   1],
       [105,   2],
       [105,   3],
       [108,  10],
       [401,   8],
       [401,   9],
       [401,  10],
       [401,  11],
       [401,  12],
       [401,  13],
       [401,  14],
       [402,   8],
       [403,   0],
       [403,   1],
       [404,   0],
       [404,   2],
       [409,   6],
       [413,  22],
       [413,  23],
       [416,   0],
       [419,  11],
       [202,  23]], dtype=int64)

In [46]:
df_copy['m_d']

0         430
1         430
2         430
3         430
4         430
5         430
6         430
7         430
8         430
9         430
10        430
11        430
12        430
13        430
14        430
15        430
16        430
17        430
18        430
19        430
20        430
21        430
22        430
23        430
24        430
25        430
26        430
27        430
28        430
29        430
         ... 
701748    101
701749    101
701750    101
701751    101
701752    101
701753    101
701754    101
701755    101
701756    101
701757    101
701758    101
701759    101
701760    101
701761    101
701762    101
701763    101
701764    101
701765    101
701766    101
701767    101
701768    101
701769    101
701770    101
701771    101
701772    101
701773    101
701774    101
701775    101
701776    101
701777    101
Name: m_d, Length: 701778, dtype: object

In [47]:
df_copy2 = df_copy.copy()

In [48]:
def get_prev(md, hr):
    m = int(md[0])
    d = int(md[1:])
    ts = pd.Timestamp(2016, m, d, hr, 0, 0)
    ts -= pd.Timedelta(hours=1)
    m_new = ts.month
    d_new = ts.day
    hr_new = ts.hour
    md_new = str(m_new)+'{:02}'.format(d_new)
    return(md_new, hr_new)

def get_next(m_d, hr):
    m = int(m_d[0])
    d = int(m_d[1:])
    ts = pd.Timestamp(2016, m, d, hr, 0, 0)
    ts += pd.Timedelta(hours=1)
    m_new = ts.month
    d_new = ts.day
    hr_new = ts.hour
    m_d_new = str(m_new)+'{:02}'.format(d_new)
    return(m_d_new, hr_new)

get_prev('105', 0)
get_next('630', 23)

('701', 0)

In [24]:
df_copy2['m_d'].head()


0    430
1    430
2    430
3    430
4    430
Name: m_d, dtype: int64

In [55]:
df_copy2['hr_temp'][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].isnull().sum()

212

In [56]:
fill_cols = ['hr_temp', 'hr_humid', 'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip']

for md, hr in nan_pairs:
    print(md, hr)
    md = str(md)
    md_prev, hr_prev = md, hr
    md_next, hr_next = md, hr
    while df_copy2['hr_temp'][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].isnull().sum() > 0:
        md_prev, hr_prev = get_prev(md_prev, hr_prev)
    while df_copy2['hr_temp'][(df_copy2['m_d'] == md_next) & (df_copy2['hr'] == hr_next)].isnull().sum() > 0:
        md_next, hr_next = get_next(md_next, hr_next)

    for col in fill_cols:
        print(col)
        col_prev = df_copy2[col][(df_copy2['m_d'] == md_prev) & (df_copy2['hr'] == hr_prev)].iloc[0]
        col_next = df_copy2[col][(df_copy2['m_d'] == md_next) & (df_copy2['hr'] == hr_next)].iloc[0]
        col_new = np.average([col_prev, col_next])
        print(col_new)
        df_copy2.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), col] = col_new
    print()


313 4
hr_temp
11.1
hr_humid
0.59
hr_dewpt
3.3
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

105 0
hr_temp
-9.7
hr_humid
0.515
hr_dewpt
-18.05
hr_windspeed
11.1
hr_gustspeed
33.3
hr_precip
0.0

105 1
hr_temp
-10.4
hr_humid
0.5375
hr_dewpt
-18.175
hr_windspeed
9.25
hr_gustspeed
33.3
hr_precip
0.0

105 2
hr_temp
-10.75
hr_humid
0.54875
hr_dewpt
-18.2375
hr_windspeed
8.325
hr_gustspeed
33.3
hr_precip
0.0

105 3
hr_temp
-10.925
hr_humid
0.554375
hr_dewpt
-18.26875
hr_windspeed
7.8625
hr_gustspeed
33.3
hr_precip
0.0

108 10
hr_temp
3.5
hr_humid
0.65
hr_dewpt
-2.5
hr_windspeed
0.0
hr_gustspeed
0.0
hr_precip
0.0

401 8
hr_temp
20.0
hr_humid
0.68
hr_dewpt
13.85
hr_windspeed
4.65
hr_gustspeed
0.0
hr_precip
0.0

401 9
hr_temp
21.1
hr_humid
0.645
hr_dewpt
14.125
hr_windspeed
2.325
hr_gustspeed
0.0
hr_precip
0.0

401 10
hr_temp
21.65
hr_humid
0.6275
hr_dewpt
14.2625
hr_windspeed
1.1625
hr_gustspeed
0.0
hr_precip
0.0

401 11
hr_temp
21.925
hr_humid
0.61875
hr_dewpt
14.33125
hr_windspeed
0.58125
h

In [57]:
df_copy2.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_lng,p_lat,d_lng,d_lat,dur,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip
0,False,430,Saturday,23,0.069424,1,-73.987793,40.724792,-73.975616,40.656445,1454,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
1,False,430,Saturday,23,0.057791,1,-73.957596,40.717770,-73.951424,40.775230,1409,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
2,False,430,Saturday,23,0.066698,2,-74.000954,40.742031,-73.947708,40.782200,1081,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
3,False,430,Saturday,23,0.018223,1,-73.985733,40.738258,-73.993179,40.754890,800,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0
4,False,430,Saturday,23,0.027198,1,-74.006615,40.740650,-73.985619,40.723362,1151,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0


In [58]:
df_copy2.to_csv('../_data/train__dropped__lnglat__saved__no_disaster_nan_fixed.csv', index=False)

In [59]:
df_copy3 = df_copy2.copy()
df_copy3.isnull().sum()

is_hol           0
m_d              0
dow              0
hr               0
L2               0
pas_cnt          0
p_lng            0
p_lat            0
d_lng            0
d_lat            0
dur              0
total_dist       0
total_time       0
day_max_temp     0
day_min_temp     0
day_rainfall     0
day_snowfall     0
day_snowdepth    0
hr_temp          0
hr_humid         0
hr_dewpt         0
hr_windspeed     0
hr_gustspeed     0
hr_precip        0
dtype: int64

In [236]:
md = '419'
hr = 11
df_copy2.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'].head()

67827    19.45
67828    19.45
67829    19.45
67830    19.45
67831    19.45
Name: hr_temp, dtype: float64

In [234]:
df_copy3.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'] = 3
df_copy3.loc[(df_copy2['m_d'] == md) & (df_copy2['hr'] == hr), 'hr_temp'].head()

67827    3.0
67828    3.0
67829    3.0
67830    3.0
67831    3.0
Name: hr_temp, dtype: float64

In [162]:
ssss[:]

67827   NaN
67828   NaN
67829   NaN
67830   NaN
67831   NaN
67832   NaN
67833   NaN
67834   NaN
67835   NaN
67836   NaN
67837   NaN
67838   NaN
67839   NaN
67840   NaN
67841   NaN
67842   NaN
67843   NaN
67844   NaN
67845   NaN
67846   NaN
67847   NaN
67848   NaN
67849   NaN
67850   NaN
67851   NaN
67852   NaN
67853   NaN
67854   NaN
67855   NaN
67856   NaN
         ..
68066   NaN
68067   NaN
68068   NaN
68069   NaN
68070   NaN
68071   NaN
68072   NaN
68073   NaN
68074   NaN
68075   NaN
68076   NaN
68077   NaN
68078   NaN
68079   NaN
68080   NaN
68081   NaN
68082   NaN
68083   NaN
68084   NaN
68085   NaN
68086   NaN
68087   NaN
68088   NaN
68089   NaN
68090   NaN
68091   NaN
68092   NaN
68093   NaN
68094   NaN
68095   NaN
Name: hr_temp, Length: 269, dtype: float64

In [ ]:
df_copy.to_csv('../_data/train__dropped__no_disaster.csv', index=False)

In [38]:
def make_hourly(timestamp):
    y = timestamp.year
    m = timestamp.month
    d = timestamp.day
    h = timestamp.hour
    return pd.Timestamp(y, m, d, h, 0, 0)

In [45]:
df_copy.insert(2, 'Time', df_copy['pickup_datetime'].apply(make_hourly))

In [44]:
del df_copy['pickup_hour']

In [46]:
df_copy.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,travel_time_per_step,step_maneuvers,step_direction,step_location_list,maximum temperature,minimum temperature,average temperature,precipitation,snow fall,snow depth
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,29.4|31.7|59|13.4|5.9|64.5|179.3|105.2|211.3|9...,depart|turn|new name|turn|turn|turn|turn|new n...,right|left|straight|right|slight right|left|sl...,"-73.987841,40.724727|-73.98987,40.725582|-73.9...",65,46,55.5,0.0,0.0,0.0
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,43.4|39.6|48.8|20.9|43.1|104|92.1|29.2|55.7|10...,depart|turn|turn|turn|new name|turn|new name|t...,none|left|left|right|straight|right|straight|l...,"-73.957606,40.717761|-73.955186,40.716252|-73....",65,46,55.5,0.0,0.0,0.0
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,197.4|155.4|27.5|0.9|35.2|10.8|32.8|135.2|20.7...,depart|turn|turn|fork|fork|fork|on ramp|merge|...,left|right|left|slight left|slight left|slight...,"-74.000886,40.742003|-73.992525,40.753478|-73....",65,46,55.5,0.0,0.0,0.0
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,7.4|12.7|129|49.8|99.1|0,depart|turn|turn|fork|turn|arrive,none|straight|right|slight right|left|arrive,"-73.985741,40.738246|-73.986427,40.738529|-73....",65,46,55.5,0.0,0.0,0.0
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,10.9|13.9|89.4|85.2|5.8|45.2|38.1|150.9|18.8|0,depart|turn|turn|turn|turn|continue|new name|t...,right|right|right|right|left|straight|straight...,"-74.006656,40.740594|-74.007473,40.740938|-74....",65,46,55.5,0.0,0.0,0.0


In [47]:
df_copy.to_csv('../_data/train_with_osrm_and_weather_h.csv')

In [42]:
df_hourly.head()

,Time,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,2015-12-31 02:00:00,7.8,0.89,6.1,NNE,5.6,0.0,0.8,None,Overcast
1,2015-12-31 03:00:00,7.2,0.90,5.6,Variable,7.4,0.0,0.3,None,Overcast
2,2015-12-31 04:00:00,7.2,0.90,5.6,Calm,0.0,0.0,0.0,None,Overcast
3,2015-12-31 05:00:00,7.2,0.86,5.0,NW,7.4,0.0,0.0,None,Overcast
4,2015-12-31 06:00:00,7.2,0.90,5.6,West,5.6,0.0,0.0,None,Overcast


In [48]:
df_merged = df_copy.merge(df_hourly, how='left', on='Time')

In [64]:
df_hourly.iloc[2901]

Time          2016-04-30 23:00:00
Temp.                        10.6
Humidity                     0.56
Dew Point                     2.2
Wind Dir                      SSE
Wind Speed                     13
Gust Speed                      0
Precip                          0
Events                       None
Conditions                  Clear
Name: 2901, dtype: object

In [49]:
df_merged.head()

,id,vendor_id,Time,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,...,snow depth,Temp.,Humidity,Dew Point,Wind Dir,Wind Speed,Gust Speed,Precip,Events,Conditions
0,id3495688,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:24,1,-73.987793,40.724792,-73.975616,40.656445,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
1,id3051282,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:22,1,-73.957596,40.717770,-73.951424,40.775230,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
2,id0082851,1,2016-04-30 23:00:00,2016-04-30 23:59:00,5/1/16 0:17,2,-74.000954,40.742031,-73.947708,40.782200,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
3,id0515725,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:12,1,-73.985733,40.738258,-73.993179,40.754890,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear
4,id1872374,2,2016-04-30 23:00:00,2016-04-30 23:58:00,5/1/16 0:17,1,-74.006615,40.740650,-73.985619,40.723362,...,0.0,10.6,0.56,2.2,SSE,13.0,0.0,0.0,None,Clear


### df + osrm + weather + hourly weather 저장

In [65]:
df_merged.to_csv('../_data/train_with_osrm_and_weather_hourly.csv')